In [35]:
holding_points = [
            ("Z1", "36L/18R", -3.573093114831562, 40.490653160130186),
            ("KA6", "32R/14L", -3.537524367869231, 40.472076292010001),
            ("KA8", "32R/14L", -3.536653485337274, 40.466622566754253),
            ("K3", "32R/14L", -3.558959606954449, 40.494122669084419),
            ("K2", "32R/14L", -3.559326044131887, 40.4945961819448),
            ("K1", "32R/14L", -3.560411408421098, 40.495554592925956),
            ("Y1", "36R/18L", -3.560656492186808, 40.499431092287409),
            ("Y2", "36R/18L", -3.560645785166937, 40.500298406173002),
            ("Y3", "36R/18L", -3.560660061193443, 40.501183565039504),
            ("Y7", "36R/18L", -3.560800449906033, 40.533391949745102),
            ("Z6", "36L/18R", -3.576129307304151, 40.495184843931881),
            ("Z4", "36L/18R", -3.576034129003182, 40.492555539298088),
            ("Z2", "36L/18R", -3.575903257941006, 40.491865496230126),
            ("Z3", "36L/18R", -3.57319305240692, 40.491819096186241),
            ("LF", "32L/14R", -3.572566658955927, 40.479721203031424),
            ("L1", "32L/14R", -3.57652786733783, 40.483565816902733),
            ("LA", "32L/14R", -3.577181028787666, 40.484251101106899),
            ("LB", "32L/14R", -3.577553413710587, 40.484873329796898),
            ("LC", "32L/14R", -3.575750378154376, 40.486690643924192),
            ("LD", "32L/14R", -3.575150753600524, 40.486522892072891),
            ("LE", "32L/14R", -3.574915186586964, 40.485580625293494)
        ]

In [47]:
def cleanScenario(df, df_answers, scenario_name):
    import pandas as pd
    import numpy as np

    df_new = df[["ts", "icao24", "df", "tc", "callsign", "bds20_callsign", "lat_deg", "lon_deg", "altitude", "bds05_altitude", "groundspeed", "bds50_groundspeed", "wake_vortex"]].copy()

    WAKE_VORTEX_TRANSLATION = {
        '<7000kg': 'Light (less than 7000 kg)',
        '<34,000kg': 'Medium 1 (between 7000 kg and 34000 kg)',
        '<136,000kg': 'Medium 2 (between 34000 kg to 136000 kg)',
        'Heavy': 'Heavy (larger than 136000 kg)',
        'High vortex': 'High vortex aircraft',
        'Rotorcraft': 'Rotorcraft',
        'Obstruction': 'Ground obstruction',
        'Surface emergency vehicle': 'Surface emergency vehicle',
    }

    df_new["TurbulenceCategory"] = df_new["wake_vortex"].map(WAKE_VORTEX_TRANSLATION).fillna(np.nan)
    df_new = df_new.drop(columns=["wake_vortex"])
    df_new["ts"] = pd.to_datetime(df_new["ts"], unit='ms')
    df_new["tc"] = df_new["tc"].astype("Int64")
    df_new["df"] = df_new["df"].astype("Int64")
    df_new["Callsign"] = df_new["callsign"].fillna(df_new["bds20_callsign"])
    df_new = df_new.drop(columns=["callsign", "bds20_callsign"])
    df_new["Speed"] = df_new["groundspeed"].fillna(df_new["bds50_groundspeed"])
    df_new = df_new.drop(columns=["groundspeed", "bds50_groundspeed"])
    df_new["Altitude (ft)"] = df_new["altitude"].fillna(df_new["bds05_altitude"])
    df_new = df_new.drop(columns=["altitude", "bds05_altitude"])

    df_new = df_new.rename(columns={
        "ts": "Timestamp (date)",
        "icao24": "ICAO",
        "df": "Downlink Format",
        "tc": "Typecode",
        "lat_deg": "lat",
        "lon_deg": "lon"
    })

    conditions_fs = [
        (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(5, 9))),
        (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(9, 23))),
    ]
    results_fs = ['on-ground', 'airborne']
    df_new["Flight status"] = np.select(conditions_fs, results_fs, default='NaN')

    # Agregar mensajes adicionales
    row = df_answers[df_answers["scenario_name"] == scenario_name].iloc[0]
    icao = row["icao"]
    holding_point = row["holding_point"]
    runway = row["runway"]

    # Buscar el punto de espera
    point_info = [p for p in holding_points if p[0] == holding_point]
    point_name, point_runway, point_lon, point_lat = point_info[0]

    df_icao = df_new[df_new["ICAO"] == icao].copy()
    df_icao["Timestamp (date)"] = pd.to_datetime(df_icao["Timestamp (date)"])  # aseguramos datetime
    last_msg = df_icao.sort_values("Timestamp (date)").iloc[-1]

    # Mensaje con velocidad cero
    msg_zero_speed = last_msg.copy()
    msg_zero_speed["Speed"] = 0
    msg_zero_speed["Timestamp (date)"] += pd.Timedelta(milliseconds=1)
    msg_zero_speed["Flight status"] = "on-ground"
    msg_zero_speed["lat"] = None
    msg_zero_speed["lon"] = None
    

    # Mensaje en punto de espera
    msg_holding = msg_zero_speed.copy()
    msg_holding["lat"] = point_lat
    msg_holding["lon"] = point_lon
    msg_holding["Speed"] = None
    msg_holding["Flight status"] = "on-ground"
    msg_holding["Timestamp (date)"] += pd.Timedelta(milliseconds=1)

    # Añadir los mensajes al DataFrame
    df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)

    df_new["Timestamp (date)"] = df_new["Timestamp (date)"].astype(str)

    print(f"Se añadieron dos mensajes sintéticos para el avión {icao} en el escenario {scenario_name}")

    return df_new


In [49]:
import pandas as pd
import os
from pathlib import Path

# Configurar para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ruta sde entrada y salida
ruta_escenarios_raw = Path("../../data/final_scenarios/raw")
ruta_escenarios_clean = Path("../../data/final_scenarios/clean")
ruta_escenarios_clean.mkdir(parents=True, exist_ok=True)

df_answers = pd.read_csv("../../data/final_scenarios/answers_empty.csv")

lastI = 0
# Iterar sobre todos los archivos .parquet en la carpeta raw
for i, parquet_file in enumerate(ruta_escenarios_raw.glob("*.parquet")):
    print(f"Procesando {parquet_file.name}")
    df = pd.read_parquet(parquet_file)
    df_clean = cleanScenario(df, df_answers=df_answers, scenario_name=parquet_file.name)
    output_path = ruta_escenarios_clean / parquet_file.name
    df_clean.to_parquet(output_path, index=False)
    print(f"Guardado en {output_path}")
    lastI = i

print(f"Total escenarios procesados: {lastI + 1}")

Procesando scenario_064.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)
/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)


Se añadieron dos mensajes sintéticos para el avión 34324e en el escenario scenario_064.parquet
Guardado en ../../data/final_scenarios/clean/scenario_064.parquet
Procesando scenario_074.parquet
Se añadieron dos mensajes sintéticos para el avión e8043a en el escenario scenario_074.parquet
Guardado en ../../data/final_scenarios/clean/scenario_074.parquet
Procesando scenario_006.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)


Se añadieron dos mensajes sintéticos para el avión 345698 en el escenario scenario_006.parquet
Guardado en ../../data/final_scenarios/clean/scenario_006.parquet
Procesando scenario_016.parquet


...

/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)


Se añadieron dos mensajes sintéticos para el avión 344217 en el escenario scenario_119.parquet
Guardado en ../../data/final_scenarios/clean/scenario_119.parquet
Procesando scenario_013.parquet


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)


Se añadieron dos mensajes sintéticos para el avión 344416 en el escenario scenario_013.parquet
Guardado en ../../data/final_scenarios/clean/scenario_013.parquet
Procesando scenario_003.parquet
Se añadieron dos mensajes sintéticos para el avión 3451d8 en el escenario scenario_003.parquet
Guardado en ../../data/final_scenarios/clean/scenario_003.parquet
Total escenarios procesados: 193


/var/folders/dp/1z2xwn7516q1b4n5t3nlr0500000gn/T/ipykernel_3467/924166058.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, pd.DataFrame([msg_zero_speed, msg_holding])], ignore_index=True)


In [13]:
df_answers

,scenario_name,icao,holding_point,runway,time_to_takeoff_s
0,scenario_001.parquet,34324e,K2,14L/32R,NaN
1,scenario_002.parquet,4bb26b,K1,14L/32R,NaN
2,scenario_003.parquet,3451d8,K3,14L/32R,NaN
3,scenario_004.parquet,3420ca,K2,14L/32R,NaN
4,scenario_005.parquet,344416,Z4,18R/36L,NaN
...,...,...,...,...,...
188,scenario_189.parquet,3445d4,Z4,18R/36L,NaN
189,scenario_190.parquet,4d251c,Z2,18R/36L,NaN
190,scenario_191.parquet,34628d,Z1,18R/36L,NaN
191,scenario_192.parquet,342107,Z4,18R/36L,NaN
